In [ ]:
import pandas as pd
import numpy as np

# load dataset from github link, create pandas dataframe
# url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/Correct_Dataset.csv'
# url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/cardio_train.csv'
url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/heart.csv'
hd_df = pd.read_csv(url)

hd_df['age'] = hd_df['age'].astype(float)
hd_df['trestbps'] = hd_df['trestbps'].astype(float)
hd_df['chol'] = hd_df['chol'].astype(float)
hd_df['thalach'] = hd_df['thalach'].astype(float)
hd_df['oldpeak'] = hd_df['oldpeak'].astype(float)

hd_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1,3,145.0,233.0,1,0,150.0,0,2.3,0,0,1,1
1,37.0,1,2,130.0,250.0,0,1,187.0,0,3.5,0,0,2,1
2,41.0,0,1,130.0,204.0,0,0,172.0,0,1.4,2,0,2,1
3,56.0,1,1,120.0,236.0,0,1,178.0,0,0.8,2,0,2,1
4,57.0,0,0,120.0,354.0,0,1,163.0,1,0.6,2,0,2,1


In [ ]:
# sklearn implementation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# decision_tree = DecisionTreeClassifier()
dataY = hd_df["target"]
dataX = hd_df.drop(["target"], axis=1)
# dataX.head()
# means = dataX.mean()
# sds = dataX.std()
# dataX = dataX.apply(lambda x: (x-means)/sds,axis=1)
# len(df)
# X_train, X_test, y_train, y_test = None, None, None, None
# best_acc, best_state = None, None
# for i in range(100):
#   X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.20, random_state=i)
#   decision_tree = DecisionTreeClassifier()
#   decision_tree.fit(X_train, y_train)
#   accuracy = decision_tree.score(X_test, y_test)
#   if best_acc is None or accuracy > best_acc:
#     best_acc = accuracy
#     best_state = i
# print(best_acc * 100, best_state)

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.20, random_state=27)
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
accuracy = decision_tree.score(X_test, y_test)
print(accuracy)

0.8360655737704918


In [ ]:
# our own decision tree implementation
import json
import copy
from pandas.api.types import is_numeric_dtype

def entropy(y):
    e = 0
    counts = y.value_counts()
    for i in counts:
        e += -1 * i/y.count() * np.log2(i/y.count())
    return e

def gain(y,x):
    g = 0
    unique_x = x.unique()
    for i in unique_x:
        filtered = x.loc[lambda item : item == i].index
        y_filtered = y.loc[filtered]
        calc_entropy = entropy(y_filtered)
        g += calc_entropy * (len(y_filtered) / len(y))

    return entropy(y) - g

def gain_ratio(y,x):
    g = gain(y, x)
    return g/entropy(y)

def select_split(X,y):
    col = None
    gr = None
    best_col = None
    best_gr = None

    if len(list(X.columns)) == 1:
        return 0, list(X.columns)[0]

    for i in X.columns:
        if is_numeric_dtype(X[i]):
            sorted_col = sorted(list(X[i].unique()))
            for j in range(len(sorted_col)-1):
                midpoint = (sorted_col[j] + sorted_col[j+1]) / 2
                curr_gr = gain_ratio(y, X[i] < midpoint)
                if best_gr is None or curr_gr > best_gr:
                    best_gr = curr_gr
                    best_col = '{}<{:.2f}'.format(i, midpoint)       
        else:
            curr_gr = gain_ratio(y, X[i])
            if best_gr is None or curr_gr > best_gr:
                best_gr = curr_gr
                best_col = i        
    col = best_col
    gr = best_gr
    return gr,col


def make_tree(X, y, min_split_count=5):
    tree = {}
    keys = list(y.value_counts().keys())
    
    if len(keys) == 1:
        return np.int64(keys[0])
    if len(X.columns) == 0 or len(X) < min_split_count:
        return y.mode()[0]

    best_split = select_split(X, y)
    curr_column = best_split[1]

    check = curr_column.find('<')
    if check != -1:
        X[curr_column] = X[curr_column[:check]] < float(curr_column[check+1:])
        X = X.drop(curr_column[:check], axis=1)
    
    curr_x = X[curr_column]
    
    if len(list(curr_x.unique())) == 1:
        return y.mode()[0]
    
    tree[curr_column] = {}
    for i in curr_x.unique():
        filtered = curr_x.loc[lambda item : item == i].index
        Y_filtered = y.loc[filtered]
        X_filtered = X.loc[filtered].drop(curr_column, axis=1)
        tree[curr_column][str(i)] = make_tree(X_filtered, Y_filtered, min_split_count)
    return tree

def print_tree(tree):
    mytree = copy.deepcopy(tree)
    def fix_keys(tree):
        if type(tree) != dict:
            if type(tree) == np.int64:
                return int(tree)
        new_tree = {}
        for key in list(tree.keys()):
            if type(key) == np.int64:
                new_tree[int(key)] = tree[key]
            else:
                new_tree[key] = tree[key]
        for key in new_tree.keys():
            new_tree[key] = fix_keys(new_tree[key])
        return new_tree
    mytree = fix_keys(mytree)
    print(json.dumps(mytree, indent=4, sort_keys=True))


def generate_rules(tree):
    rules = []
    if type(tree) == np.int64:
        return [[tree]]
    
    root = list(tree.keys())[0]
    for key, val in tree[root].items():
        res = generate_rules(val)
        for i in range(len(res)):
            res[i].insert(0, (root, key))
            rules.append(res[i])
    return rules


def make_prediction(rules,x,default):
    for i in range(len(rules)):
        did_find = True
            
        for j in range(len(rules[i])):
            if rules[i][j] == 0 or rules[i][j] == 1:
                break
            curr_item = rules[i][j][0].find('<')
            if curr_item != -1:
                name = rules[i][j][0][:curr_item]
                number = float(rules[i][j][0][curr_item+1:])
                if (x[name] > number and rules[i][j][1] == "True") or (x[name] < number and rules[i][j][1] == "False"):
                    did_find = False
                    break
            else:
                if x[rules[i][j][0]] != rules[i][j][1]:
                    did_find = False
                    break
        if did_find:
            return rules[i][-1]
    return(default)




In [ ]:
curr_tree = make_tree(X_train, y_train, 10)
print_tree(curr_tree)

{
    "cp<0.50": {
        "False": {
            "thal<2.50": {
                "False": {
                    "thalach<132.50": {
                        "False": {
                            "chol<205.50": {
                                "False": {
                                    "age<67.50": {
                                        "False": 0,
                                        "True": {
                                            "sex<0.50": {
                                                "False": {
                                                    "fbs<0.50": {
                                                        "False": 1,
                                                        "True": {
                                                            "ca<2.50": {
                                                                "False": 1,
                                                                "True": {
                                                    

In [ ]:
rules = generate_rules(curr_tree)
for i in range(10):
  print(rules[i])
default = 0
y_c45 = X_test.apply(lambda x: make_prediction(rules,x,default),axis=1)

count = 0
for i in y_test.index:
  if y_test.loc[i] == y_c45.loc[i]:
    count += 1

print('accuracy:', count/len(y_test))

[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'False'), ('sex<0.50', 'False'), ('chol<245.50', 'True'), ('ca<0.50', 'True'), 1]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'False'), ('sex<0.50', 'False'), ('chol<245.50', 'True'), ('ca<0.50', 'False'), 0]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'False'), ('sex<0.50', 'False'), ('chol<245.50', 'False'), 0]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'False'), ('sex<0.50', 'True'), ('trestbps<138.00', 'False'), 1]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'False'), ('sex<0.50', 'True'), ('trestbps<138.00', 'True'), 1]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'True'), ('thalach<152.50', 'True'), ('sex<0.50', 'True'), 1]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'True'), ('thalach<152.50', 'True'), ('sex<0.50', 'False'), 1]
[('cp<0.50', 'False'), ('thal<2.50', 'True'), ('age<55.50', 'True'), ('thalach<152.50', 'False'), 1

In [ ]:
def feature_importance(Xtrain,ytrain,Xtest,ytest, npermutations = 5):
    # initialize what we are going to return
    importances = {}
    for col in Xtrain.columns:
        importances[col] = 0
    # find the original accuracy
    curr_tree = make_tree(Xtrain, ytrain)
    rules = generate_rules(curr_tree)
    orig_accuracy = make_prediction(rules,x,default)
    # now carray out the feature importance work
    for col in Xtrain.columns:
        for perm in range(npermutations):
            Xtest2 = Xtest.copy()
            Xtest2[col] = Xtest[col].sample(frac=1, replace=False).values
            curr_tree = make_tree(Xtrain, ytrain)
            rules = generate_rules(curr_tree)
            error = orig_accuracy - make_prediction(Xtrain,Xtest2,ytrain,ytest)
            importances[col] += error
            
        importances[col] = importances[col]/npermutations

    return importances

feature_importance(X_train,y_train,X_test,y_test)

NameError: ignored